In [1]:
import pandas as pd
import numpy as np
from io import StringIO
import re

import random
from datetime import datetime
from datetime import timedelta
import time
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Read in data set; change separators to '|' instead of ',' so that node lists don't get cut off

In [3]:
for_pd = StringIO()
with open('../data/accre-jobs-2020.csv') as accre:
    for line in accre:
        new_line = re.sub(r',', '|', line.rstrip(), count=12)
        print (new_line, file=for_pd)

for_pd.seek(0)

accre = pd.read_csv(for_pd, sep='|')[1000000:1005000]
print (accre)

            JOBID ACCOUNT      USER   REQMEM    USEDMEM     REQTIME    USEDTIME  NODES  CPUS   PARTITION EXITCODE      STATE NODELIST
1000000  18616746     cms  cmslocal   5000Mn   1284.14M  2-00:00:00    00:15:35      1     1  production      0:0  COMPLETED   cn1423
1000001  18616752     cms  cmslocal   5000Mn   1275.16M  2-00:00:00    00:11:41      1     1  production      0:0  COMPLETED   cn1424
1000002  18616754     cms  cmslocal   5000Mn   1259.60M  2-00:00:00    00:15:35      1     1  production      0:0  COMPLETED   cn1311
1000003  18616757     cms  cmslocal   5000Mn   1271.18M  2-00:00:00    00:15:36      1     1  production      0:0  COMPLETED   cn1311
1000004  18616758     cms  cmslocal   5000Mn   1260.83M  2-00:00:00    00:15:35      1     1  production      0:0  COMPLETED    cn372
...           ...     ...       ...      ...        ...         ...         ...    ...   ...         ...      ...        ...      ...
1004995  18625828     cms  cmspilot  28000Mn  11730.53M  2-00:

In [4]:
accre.tail()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
1004995,18625828,cms,cmspilot,28000Mn,11730.53M,2-00:00:00,1-18:30:30,1,4,production,0:0,COMPLETED,cn1212
1004996,18625829,cms,cmspilot,28000Mn,4993.34M,2-00:00:00,03:02:21,1,4,production,0:0,COMPLETED,cn1215
1004997,18625830,cms,cmspilot,28000Mn,9823.94M,2-00:00:00,16:47:29,1,4,production,0:0,COMPLETED,cn1216
1004998,18625831,cms,cmspilot,28000Mn,5074.64M,2-00:00:00,06:07:29,1,4,production,0:0,COMPLETED,cn1217
1004999,18625832,cms,cmspilot,28000Mn,9819.73M,2-00:00:00,23:22:20,1,4,production,0:0,COMPLETED,cn1205


In [5]:
accre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 1000000 to 1004999
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   JOBID      5000 non-null   object
 1   ACCOUNT    5000 non-null   object
 2   USER       5000 non-null   object
 3   REQMEM     5000 non-null   object
 4   USEDMEM    5000 non-null   object
 5   REQTIME    5000 non-null   object
 6   USEDTIME   5000 non-null   object
 7   NODES      5000 non-null   int64 
 8   CPUS       5000 non-null   int64 
 9   PARTITION  5000 non-null   object
 10  EXITCODE   5000 non-null   object
 11  STATE      5000 non-null   object
 12  NODELIST   5000 non-null   object
dtypes: int64(2), object(11)
memory usage: 507.9+ KB


In [6]:
accre.describe

<bound method NDFrame.describe of             JOBID ACCOUNT      USER   REQMEM    USEDMEM     REQTIME    USEDTIME  NODES  CPUS   PARTITION EXITCODE      STATE NODELIST
1000000  18616746     cms  cmslocal   5000Mn   1284.14M  2-00:00:00    00:15:35      1     1  production      0:0  COMPLETED   cn1423
1000001  18616752     cms  cmslocal   5000Mn   1275.16M  2-00:00:00    00:11:41      1     1  production      0:0  COMPLETED   cn1424
1000002  18616754     cms  cmslocal   5000Mn   1259.60M  2-00:00:00    00:15:35      1     1  production      0:0  COMPLETED   cn1311
1000003  18616757     cms  cmslocal   5000Mn   1271.18M  2-00:00:00    00:15:36      1     1  production      0:0  COMPLETED   cn1311
1000004  18616758     cms  cmslocal   5000Mn   1260.83M  2-00:00:00    00:15:35      1     1  production      0:0  COMPLETED    cn372
...           ...     ...       ...      ...        ...         ...         ...    ...   ...         ...      ...        ...      ...
1004995  18625828     cms  c

In [7]:
accre.PARTITION.value_counts()

production    4402
nogpfs         492
sam             78
pascal          22
maxwell          6
Name: PARTITION, dtype: int64

In [8]:
accre.STATE.value_counts()

COMPLETED    4997
CANCELLED       3
Name: STATE, dtype: int64

Separate memory amount from memory units

In [9]:
test_split = accre.loc[1000000,'REQMEM']
test_split

'5000Mn'

In [10]:
re.findall(r'..$', test_split)
# also re.findall(r'\D+', test_split)

['Mn']

In [11]:
re.findall(r'\d+', test_split)

['5000']

Extract req memory quantity only and put it in a new column

In [12]:
accre['REQMEM_NO'] = accre['REQMEM'].str.extract('(\d+)')

In [13]:
accre.head(100)

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO
1000000,18616746,cms,cmslocal,5000Mn,1284.14M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1423,5000
1000001,18616752,cms,cmslocal,5000Mn,1275.16M,2-00:00:00,00:11:41,1,1,production,0:0,COMPLETED,cn1424,5000
1000002,18616754,cms,cmslocal,5000Mn,1259.60M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1311,5000
1000003,18616757,cms,cmslocal,5000Mn,1271.18M,2-00:00:00,00:15:36,1,1,production,0:0,COMPLETED,cn1311,5000
1000004,18616758,cms,cmslocal,5000Mn,1260.83M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn372,5000
1000005,18616762,cms,cmslocal,5000Mn,1258.38M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn361,5000
1000006,18616763,cmsadmin,autocms,2048Mc,828.82M,12:00:00,00:05:08,1,1,production,0:0,COMPLETED,cn1122,2048
1000007,18616858,cms,cmspilot,28000Mn,2846.65M,2-00:00:00,01:12:11,1,4,production,0:0,COMPLETED,cn1299,28000
1000008,18616859,cms,cmspilot,28000Mn,9838.76M,2-00:00:00,1-22:25:09,1,4,production,0:0,COMPLETED,cn1299,28000
1000009,18616860,cms,cmspilot,28000Mn,4915.84M,2-00:00:00,20:12:20,1,4,production,0:0,COMPLETED,cn1356,28000


Extract req memory units (type) only and put it in a new column

In [14]:
accre['REQMEM_TYPE'] = accre['REQMEM'].str.extract('(\D+)')

In [15]:
# convert column values to numeric values
accre['REQMEM_NO'] = pd.to_numeric(accre['REQMEM_NO'])

In [16]:
# isolate Mc req memory units only
accre[accre['REQMEM_TYPE'] == 'Mc']

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO,REQMEM_TYPE
1000006,18616763,cmsadmin,autocms,2048Mc,828.82M,12:00:00,00:05:08,1,1,production,0:0,COMPLETED,cn1122,2048,Mc
1000054,18616942,cmsadmin,autocms,2048Mc,860.54M,12:00:00,00:05:25,1,1,production,0:0,COMPLETED,cn430,2048,Mc
1000165,18617189,pasilla,earlean,3072Mc,197.12M,2-00:00:00,00:03:07,1,10,production,0:15,CANCELLED,cn1459,3072,Mc
1000166,18617190,cmsadmin,autocms,2048Mc,817.76M,12:00:00,00:02:42,1,1,production,0:0,COMPLETED,cn1133,2048,Mc
1000203,18617244_1,casaba,anne,8192Mc,1010.52M,02:30:00,02:30:15,1,1,production,0:0,COMPLETED,cn1335,8192,Mc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004883,18625524_97,cep,gemma,5120Mc,0,1-15:00:00,00:00:05,1,1,production,0:0,COMPLETED,cn1261,5120,Mc
1004884,18625524_98,cep,gemma,5120Mc,0,1-15:00:00,00:00:05,1,1,production,0:0,COMPLETED,cn1261,5120,Mc
1004885,18625524_99,cep,gemma,5120Mc,0,1-15:00:00,00:00:04,1,1,production,0:0,COMPLETED,cn1308,5120,Mc
1004887,18625526,cmsadmin,autocms,2048Mc,811.26M,12:00:00,00:03:01,1,1,production,0:0,COMPLETED,cn1084,2048,Mc


In [17]:
accre.head(10)

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO,REQMEM_TYPE
1000000,18616746,cms,cmslocal,5000Mn,1284.14M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1423,5000,Mn
1000001,18616752,cms,cmslocal,5000Mn,1275.16M,2-00:00:00,00:11:41,1,1,production,0:0,COMPLETED,cn1424,5000,Mn
1000002,18616754,cms,cmslocal,5000Mn,1259.60M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1311,5000,Mn
1000003,18616757,cms,cmslocal,5000Mn,1271.18M,2-00:00:00,00:15:36,1,1,production,0:0,COMPLETED,cn1311,5000,Mn
1000004,18616758,cms,cmslocal,5000Mn,1260.83M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn372,5000,Mn
1000005,18616762,cms,cmslocal,5000Mn,1258.38M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn361,5000,Mn
1000006,18616763,cmsadmin,autocms,2048Mc,828.82M,12:00:00,00:05:08,1,1,production,0:0,COMPLETED,cn1122,2048,Mc
1000007,18616858,cms,cmspilot,28000Mn,2846.65M,2-00:00:00,01:12:11,1,4,production,0:0,COMPLETED,cn1299,28000,Mn
1000008,18616859,cms,cmspilot,28000Mn,9838.76M,2-00:00:00,1-22:25:09,1,4,production,0:0,COMPLETED,cn1299,28000,Mn
1000009,18616860,cms,cmspilot,28000Mn,4915.84M,2-00:00:00,20:12:20,1,4,production,0:0,COMPLETED,cn1356,28000,Mn


In [18]:
# testing out math for memory conversions
test_mn = accre.loc[1000000, ['REQMEM_NO', 'REQMEM', 'NODES', 'CPUS', 'REQMEM_TYPE']]
test_mn['CPUS'] = pd.to_numeric(1)
test_mn

REQMEM_NO        5000
REQMEM         5000Mn
NODES               1
CPUS                1
REQMEM_TYPE        Mn
Name: 1000000, dtype: object

In [19]:
test_mc = accre.loc[1000006, ['REQMEM_NO', 'REQMEM', 'NODES', 'CPUS', 'REQMEM_TYPE']]
test_mc['CPUS'] = pd.to_numeric(1)
test_mc

REQMEM_NO        2048
REQMEM         2048Mc
NODES               1
CPUS                1
REQMEM_TYPE        Mc
Name: 1000006, dtype: object

In [20]:
if test_mc['REQMEM_TYPE'] == 'Mc':
    print(test_mc['REQMEM_NO'])
elif test_mc['REQMEM_TYPE'] == 'Mn':
    print(test_mc['REQMEM_NO'] * (test_mc['NODES']/test_mc['CPUS']))

2048


In [21]:
if test_mn['REQMEM_TYPE'] == 'Mc':
    print(test_mn['REQMEM_NO'])
elif test_mn['REQMEM_TYPE'] == 'Mn':
    print(test_mn['REQMEM_NO'] * (test_mn['NODES']/test_mn['CPUS']))

5000.0


In [22]:
accre['REQMEM_PER_CORE'] = np.where((accre['REQMEM_TYPE'] == 'Mn'), \
                                      (accre['REQMEM_NO'] * (accre['NODES']/accre['CPUS'])), \
                                       accre['REQMEM_NO'])

accre.head(50)

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO,REQMEM_TYPE,REQMEM_PER_CORE
1000000,18616746,cms,cmslocal,5000Mn,1284.14M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1423,5000,Mn,5000.000
1000001,18616752,cms,cmslocal,5000Mn,1275.16M,2-00:00:00,00:11:41,1,1,production,0:0,COMPLETED,cn1424,5000,Mn,5000.000
1000002,18616754,cms,cmslocal,5000Mn,1259.60M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1311,5000,Mn,5000.000
1000003,18616757,cms,cmslocal,5000Mn,1271.18M,2-00:00:00,00:15:36,1,1,production,0:0,COMPLETED,cn1311,5000,Mn,5000.000
1000004,18616758,cms,cmslocal,5000Mn,1260.83M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn372,5000,Mn,5000.000
1000005,18616762,cms,cmslocal,5000Mn,1258.38M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn361,5000,Mn,5000.000
1000006,18616763,cmsadmin,autocms,2048Mc,828.82M,12:00:00,00:05:08,1,1,production,0:0,COMPLETED,cn1122,2048,Mc,2048.000
1000007,18616858,cms,cmspilot,28000Mn,2846.65M,2-00:00:00,01:12:11,1,4,production,0:0,COMPLETED,cn1299,28000,Mn,7000.000
1000008,18616859,cms,cmspilot,28000Mn,9838.76M,2-00:00:00,1-22:25:09,1,4,production,0:0,COMPLETED,cn1299,28000,Mn,7000.000
1000009,18616860,cms,cmspilot,28000Mn,4915.84M,2-00:00:00,20:12:20,1,4,production,0:0,COMPLETED,cn1356,28000,Mn,7000.000


In [23]:
accre['USEDMEM_TYPE'] = accre['USEDMEM'].str.extract('(\D$)')

In [24]:
accre['USEDMEM_NO'] = accre['USEDMEM'].str.extract('(\d+)')

In [25]:
accre.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO,REQMEM_TYPE,REQMEM_PER_CORE,USEDMEM_TYPE,USEDMEM_NO
1000000,18616746,cms,cmslocal,5000Mn,1284.14M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1423,5000,Mn,5000.0,M,1284
1000001,18616752,cms,cmslocal,5000Mn,1275.16M,2-00:00:00,00:11:41,1,1,production,0:0,COMPLETED,cn1424,5000,Mn,5000.0,M,1275
1000002,18616754,cms,cmslocal,5000Mn,1259.60M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1311,5000,Mn,5000.0,M,1259
1000003,18616757,cms,cmslocal,5000Mn,1271.18M,2-00:00:00,00:15:36,1,1,production,0:0,COMPLETED,cn1311,5000,Mn,5000.0,M,1271
1000004,18616758,cms,cmslocal,5000Mn,1260.83M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn372,5000,Mn,5000.0,M,1260


In [26]:
accre['USEDMEM_NO'] = pd.to_numeric(accre['USEDMEM_NO'])

In [27]:
accre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 1000000 to 1004999
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   JOBID            5000 non-null   object 
 1   ACCOUNT          5000 non-null   object 
 2   USER             5000 non-null   object 
 3   REQMEM           5000 non-null   object 
 4   USEDMEM          5000 non-null   object 
 5   REQTIME          5000 non-null   object 
 6   USEDTIME         5000 non-null   object 
 7   NODES            5000 non-null   int64  
 8   CPUS             5000 non-null   int64  
 9   PARTITION        5000 non-null   object 
 10  EXITCODE         5000 non-null   object 
 11  STATE            5000 non-null   object 
 12  NODELIST         5000 non-null   object 
 13  REQMEM_NO        5000 non-null   int64  
 14  REQMEM_TYPE      5000 non-null   object 
 15  REQMEM_PER_CORE  5000 non-null   float64
 16  USEDMEM_TYPE     4193 non-null   object 
 17  USEDM

In [28]:
accre['USEDMEM_PER_CORE'] = accre['USEDMEM_NO'] / accre['CPUS']

In [29]:
accre.head(10)

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO,REQMEM_TYPE,REQMEM_PER_CORE,USEDMEM_TYPE,USEDMEM_NO,USEDMEM_PER_CORE
1000000,18616746,cms,cmslocal,5000Mn,1284.14M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1423,5000,Mn,5000.0,M,1284,1284.00
1000001,18616752,cms,cmslocal,5000Mn,1275.16M,2-00:00:00,00:11:41,1,1,production,0:0,COMPLETED,cn1424,5000,Mn,5000.0,M,1275,1275.00
1000002,18616754,cms,cmslocal,5000Mn,1259.60M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1311,5000,Mn,5000.0,M,1259,1259.00
1000003,18616757,cms,cmslocal,5000Mn,1271.18M,2-00:00:00,00:15:36,1,1,production,0:0,COMPLETED,cn1311,5000,Mn,5000.0,M,1271,1271.00
1000004,18616758,cms,cmslocal,5000Mn,1260.83M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn372,5000,Mn,5000.0,M,1260,1260.00
1000005,18616762,cms,cmslocal,5000Mn,1258.38M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn361,5000,Mn,5000.0,M,1258,1258.00
1000006,18616763,cmsadmin,autocms,2048Mc,828.82M,12:00:00,00:05:08,1,1,production,0:0,COMPLETED,cn1122,2048,Mc,2048.0,M,828,828.00
1000007,18616858,cms,cmspilot,28000Mn,2846.65M,2-00:00:00,01:12:11,1,4,production,0:0,COMPLETED,cn1299,28000,Mn,7000.0,M,2846,711.50
1000008,18616859,cms,cmspilot,28000Mn,9838.76M,2-00:00:00,1-22:25:09,1,4,production,0:0,COMPLETED,cn1299,28000,Mn,7000.0,M,9838,2459.50
1000009,18616860,cms,cmspilot,28000Mn,4915.84M,2-00:00:00,20:12:20,1,4,production,0:0,COMPLETED,cn1356,28000,Mn,7000.0,M,4915,1228.75


In [30]:
# time required and used; conversion to seconds
# Create a function to split the hh:mm:ss string and calculate seconds from it
def to_sec(x):
    h,m,s = map(int,x.split(':'))
    return (h*60+m)*60+s

In [31]:
# Separate out the days from REQTIME, put them in another column, convert to a number, fill NaN with 0, convert to seconds
accre['REQTIME_DAY_SEC'] = accre['REQTIME'].str.extract('(\d+)-')
accre['REQTIME_DAY_SEC'] = pd.to_numeric(accre['REQTIME_DAY_SEC'])
accre['REQTIME_DAY_SEC'] = accre['REQTIME_DAY_SEC'].fillna(0)
accre['REQTIME_DAY_SEC'] = accre['REQTIME_DAY_SEC']*24*60*60
# Extract the hh:mm:ss from REQTIME, put in a new column, and then apply the to_sec function 
accre['REQTIME_T'] = accre['REQTIME'].str.extract('(..:..:..)$')
# REQTIME_SEC includes total seconds from REQTIME
accre['REQTIME_SEC'] = accre['REQTIME_T'].apply(to_sec) + accre['REQTIME_DAY_SEC']
# Do the same for USEDTIME
accre['USEDTIME_DAY_SEC'] = accre['USEDTIME'].str.extract('(\d+)-')
accre['USEDTIME_DAY_SEC'] = pd.to_numeric(accre['USEDTIME_DAY_SEC'])
accre['USEDTIME_DAY_SEC'] = accre['USEDTIME_DAY_SEC'].fillna(0)
accre['USEDTIME_DAY_SEC'] = accre['USEDTIME_DAY_SEC']*24*60*60
# Do the same for USEDTIME
accre['USEDTIME_T'] = accre['USEDTIME'].str.extract('(..:..:..)$')
# USEDTIME_SEC includes total second from USEDTIME
accre['USEDTIME_SEC'] = accre['USEDTIME_T'].apply(to_sec) + accre['USEDTIME_DAY_SEC']
# Check to make sure the data types look okay
accre.head(20)

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,REQMEM_NO,REQMEM_TYPE,REQMEM_PER_CORE,USEDMEM_TYPE,USEDMEM_NO,USEDMEM_PER_CORE,REQTIME_DAY_SEC,REQTIME_T,REQTIME_SEC,USEDTIME_DAY_SEC,USEDTIME_T,USEDTIME_SEC
1000000,18616746,cms,cmslocal,5000Mn,1284.14M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1423,5000,Mn,5000.000,M,1284,1284.0000,172800.0,00:00:00,172800.0,0.0,00:15:35,935.0
1000001,18616752,cms,cmslocal,5000Mn,1275.16M,2-00:00:00,00:11:41,1,1,production,0:0,COMPLETED,cn1424,5000,Mn,5000.000,M,1275,1275.0000,172800.0,00:00:00,172800.0,0.0,00:11:41,701.0
1000002,18616754,cms,cmslocal,5000Mn,1259.60M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn1311,5000,Mn,5000.000,M,1259,1259.0000,172800.0,00:00:00,172800.0,0.0,00:15:35,935.0
1000003,18616757,cms,cmslocal,5000Mn,1271.18M,2-00:00:00,00:15:36,1,1,production,0:0,COMPLETED,cn1311,5000,Mn,5000.000,M,1271,1271.0000,172800.0,00:00:00,172800.0,0.0,00:15:36,936.0
1000004,18616758,cms,cmslocal,5000Mn,1260.83M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn372,5000,Mn,5000.000,M,1260,1260.0000,172800.0,00:00:00,172800.0,0.0,00:15:35,935.0
1000005,18616762,cms,cmslocal,5000Mn,1258.38M,2-00:00:00,00:15:35,1,1,production,0:0,COMPLETED,cn361,5000,Mn,5000.000,M,1258,1258.0000,172800.0,00:00:00,172800.0,0.0,00:15:35,935.0
1000006,18616763,cmsadmin,autocms,2048Mc,828.82M,12:00:00,00:05:08,1,1,production,0:0,COMPLETED,cn1122,2048,Mc,2048.000,M,828,828.0000,0.0,12:00:00,43200.0,0.0,00:05:08,308.0
1000007,18616858,cms,cmspilot,28000Mn,2846.65M,2-00:00:00,01:12:11,1,4,production,0:0,COMPLETED,cn1299,28000,Mn,7000.000,M,2846,711.5000,172800.0,00:00:00,172800.0,0.0,01:12:11,4331.0
1000008,18616859,cms,cmspilot,28000Mn,9838.76M,2-00:00:00,1-22:25:09,1,4,production,0:0,COMPLETED,cn1299,28000,Mn,7000.000,M,9838,2459.5000,172800.0,00:00:00,172800.0,86400.0,22:25:09,167109.0
1000009,18616860,cms,cmspilot,28000Mn,4915.84M,2-00:00:00,20:12:20,1,4,production,0:0,COMPLETED,cn1356,28000,Mn,7000.000,M,4915,1228.7500,172800.0,00:00:00,172800.0,0.0,20:12:20,72740.0


In [32]:
accre.to_csv(r'../data/accre-jobs-2020-clean.csv', index = False)

In [ ]:
# breaking up node lists
nodelist = failed['NODELIST'].unique().tolist()
nodelist_nums = [i.strip('cn').strip('[').strip(']') for i in nodelist]
for n in nodelist_nums:
    if ',' in n: 
        nodes = n.split(',')
        if '-' in n:
            nodes = n.split('-')
            low, high = int(nodes[0]), int(nodes[1])       
        else:
            nodes = int(n)
